# Global sensitivity analysis applied to ordinary differential equation model
Simon Frost (@sdwfrost), 2022-03-01

## Introduction

The classical ODE version of the SIR model is:

- Deterministic
- Continuous in time
- Continuous in state

This tutorial uses tools from the `GlobalSensitivity.jl` package to explore sensitivity of the peak number of infected individuals, the timing of the peak and the final size of the epidemic to changes in parameter values.

## Libraries

In [ ]:
using OrdinaryDiffEq
using DiffEqCallbacks
using GlobalSensitivity
using Distributions
using Plots

## Transitions

The following function provides the derivatives of the model, which it changes in-place. State variables and parameters are unpacked from `u` and `p`.

In [ ]:
function sir_ode!(du,u,p,t)
    (S,I,R) = u
    (β,c,γ) = p
    N = S+I+R
    @inbounds begin
        du[1] = -β*c*I/N*S
        du[2] = β*c*I/N*S - γ*I
        du[3] = γ*I
    end
    nothing
end;

## Time domain

We set the timespan for simulations, `tspan`, initial conditions, `u0`, and parameter values, `p`. We will set the maximum time to be high, as we will be using a callback in order to stop the integration early.

In [ ]:
tmax = 10000.0
tspan = (0.0,tmax);

## Callbacks

If we just wanted the final size, we could use a `SteadyStateProblem` with the `DynamicSS` solver. To get access to the entire solution, we can use a callback instead to stop the simulation when it reaches a steady state.

In [ ]:
cb_ss = TerminateSteadyState();

## Initial conditions and parameter values

We first set fixed parameters, in this case, the total population size, `N`. In addition, in order to define an `ODEProblem`, we also need a default set of initial conditions, `u`, and parameter values, `p`.

In [ ]:
N = 1000.0;
u0 = [990.0,10.0,0.0];
p = [0.05,10.0,0.25]; # β,c,γ

We specify lower (`lb`) and upper (`ub`) bounds for each parameter.

In [ ]:
n_samples = 1000 # Number of samples
# Parameters are β, c, γ, I₀
lb = [0.01, 5.0, 0.1, 1.0]
ub = [0.1, 20.0, 1.0, 50.0]
n_params = 4;

## Running the model

In [ ]:
prob_ode = ODEProblem(sir_ode!,u0,tspan,p);

We will consider three summary statistics of the simulation for sensitivity analysis:

1. The peak number of infected individuals, `I`.
2. The time at which the peak is reached.
3. The final size of the epidemic (as `R(0)=0`, this will be `R(t_stop)` where `t_stop` is the time at which the steady state is reached).

## Serial

In the serial implementation, we write a function that takes a `Vector` of parameters and initial conditions and returns a `Vector` of outputs.

In [ ]:
f1 = function(pu0)
  p = pu0[1:3]
  I0 = pu0[4]
  u0 = [N-I0,I0,0.0]
  prob = remake(prob_ode;p=p,u=u0)
  sol = solve(prob, ROS34PW3(),callback=cb_ss)
  [maximum(sol[2,:]), sol.t[argmax(sol[2,:])], sol[end][3]]
end;

### Morris method

In [ ]:
m_morris = gsa(f1, Morris(num_trajectory=n_samples), [[lb[i],ub[i]] for i in 1:n_params]);

In [ ]:
m_morris.means

In [ ]:
m_morris.variances

### Sobol

In [ ]:
m_sobol = gsa(f1, Sobol(), [[lb[i],ub[i]] for i in 1:n_params],N=n_samples);

In [ ]:
m_sobol.ST

In [ ]:
m_sobol.S1

### Regression

In [ ]:
m_regression = gsa(f1, RegressionGSA(rank=true), [[lb[i],ub[i]] for i in 1:n_params]; samples = n_samples);

In [ ]:
m_regression.partial_correlation

In [ ]:
m_regression.partial_rank_correlation

### eFAST

In [ ]:
m_efast = gsa(f1, eFAST(), [[lb[i],ub[i]] for i in 1:n_params]; n = n_samples);

In [ ]:
m_efast.ST

In [ ]:
m_efast.S1

## Running the model in parallel

To run the above in parallel, we pass a `Matrix` of parameter values and use `EnsembleProblem` internally in order to run different parameter sets in parallel, returning a `Matrix` of outputs. The function below uses threads to parallelize, but it can easily be adapted to other modes of parallelism.

In [ ]:
pf1 = function (pu0)
  p = pu0[1:3,:]
  I0 = pu0[4,:]
  prob_func(prob,i,repeat) = remake(prob;p=p[:,i],u=[N-I0[i],I0[i],0.0])
  ensemble_prob = EnsembleProblem(prob_ode,prob_func=prob_func)
  sol = solve(ensemble_prob,ROS34PW3(),EnsembleThreads();trajectories=size(p,2))
  out = zeros(3,size(p,2))
  for i in 1:size(p,2)
    out[1,i] = maximum(sol[i][2,:])
    out[2,i] = sol[i].t[argmax(sol[i][2,:])]
    out[3,i] = sol[i][end][3]
  end
  out
end;

We then pass the keyword `batch=true` to `gsa`; here is an example for the `eFAST` method.

In [ ]:
m_efast_parallel = gsa(pf1, eFAST(), [[lb[i],ub[i]] for i in 1:n_params]; n = n_samples, batch = true);